In [ ]:
%matplotlib widget

# SANS data reduction

This notebook will guide you through the data reduction for the SANS experiment that you simulated with McStas yesterday.

The following is a basic outline of what this notebook will cover:

- Loading the NeXus files that contain the data
- Inspect/visualize the data contents
- How to convert the raw `time-of-flight` coordinate to something more useful ($\lambda$, $Q$, ...)
- Normalize to a flat field run
- Write the results to file
- Process more than one pulse

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp
import utils

## Process the run with a sample

### Load the NeXus file data

In [ ]:
folder = "../3-mcstas/SANS_with_sample_1_pulse"

sample = utils.load_sans(folder)

The first way to inspect the data is to view the HTML representation of the loaded object.

Try to explore what is inside the data, and familiarize yourself with the different sections (`Dimensions`, `Coordinates`, `Data`).

In [ ]:
sample

### Visualize the data

Here is a 2D visualization of the neutron counts, histogrammed along the `tof` and `y` dimensions:

In [ ]:
sample.hist(tof=200, y=200).plot(norm="log", vmin=1.0e-2)

Histogramming along `y` only gives a 1D plot:

In [ ]:
sample.hist(y=200).plot(norm="log")

### Coordinate transformations

The first step in the data reduction is to convert the raw event coordinates (position, time-of-flight) to something physically meaningful such as wavelength ($\lambda$) or momentum transfer ($Q$).

Scipp has a dedicated method for this called `transform_coords` (see docs [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)).

We begin with a standard graph which describes how to compute e.g. the wavelength from the other coordinates in the raw data.

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import kinematic

graph = {**beamline(scatter=True), **kinematic("tof")}
sc.show_graph(graph, simplified=True)

To compute the wavelength of all the events, we simply call `transform_coords` on our loaded data,
requesting the name of the coordinate we want in the output (`"wavelength"`),
as well as providing it the graph to be used to compute it (i.e. the one we defined just above).

This yields

In [ ]:
sample_wav = sample.transform_coords("wavelength", graph=graph)
sample_wav

The result has a `wavelength` coordinate. We can also plot the result:

In [ ]:
sample_wav.hist(wavelength=200).plot()

We can see that the range of observed wavelengths agrees with the range set in the McStas model (5.25 - 6.75 Å)

### Exercise 1: convert raw data to $Q$

Instead of wavelength as in the example above,
the task is now to convert the raw data to momentum-transfer $Q$.

The transformation graph is missing the computation for $Q$ so you will have to add it in yourself.
As a reminder, $Q$ is computed as follows

$$Q = \frac{4\pi \sin(\theta)}{\lambda}$$

You have to:

- create a function that computes $Q$
- add it to the graph
- call `transform_coords` using the new graph

**Solution:**

In [ ]:
# Insert your solution:


### Histogram the data in $Q$

The final step in processing the sample run is to histogram the data into $Q$ bins.

In [ ]:
sample_h = sample_q.hist(Q=200)
sample_h.plot(norm="log", vmin=1)

The histogrammed data currently has no standard deviations on the counts.
This needs to be added after we have performed the histogramming operation.

When dealing with neutron events, we assume the data has a Poisson distribution.
This means that the variance in a bin is equal to the counts in that bin
(the standard deviation is then $\sigma = \sqrt{\mathrm{counts}}$).

We provide a helper function that will add Poisson variances to any given input:

In [ ]:
utils.add_variances(sample_h)
sample_h.data

In [ ]:
sample_h.plot(norm="log", vmin=1)

## Exercise 2: process flat-field run

Repeat the step carried out above for the run that contained no sample (also known as a "flat-field" run).

**Solution:**

In [ ]:
# Insert your solution:


**Bonus question:** can you explain why the counts in the flat-field data drop at high $Q$?

## Exercise 3: Normalize the sample run

The flat-field run is giving a measure of the efficiency of each detector pixel.
This efficiency now needs to be used to correct the counts in the sample run to yield a realistic $I(Q)$ profile.

In particular, this should remove any unwanted artifacts in the data,
such as the drop in counts around 0.03 Å<sup>-1</sup> due to the air bubble inside the detector tube.

Normalizing is essentially just dividing the sample run by the flat field run.

**Hint:** you may run into an error like `"Mismatch in coordinate 'Q'"`. Why is this happening? How can you get around it?

**Solution:**

In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


In [ ]:
normed.plot(norm="log", vmin=1.0e-3, vmax=10.0)

## Save result to disk

Finally, we need to save our results to disk,
so that the reduced data can be forwarded to the next step in the pipeline (data analysis).

We will use a simple text file for this:

In [ ]:
from scippneutron.io import save_xye

# The simple file format does not support bin-edge coordinates.
# So we convert to bin-centers first.
data = normed.copy()
data.coords["Q"] = sc.midpoints(data.coords["Q"])

save_xye("sans_iofq.dat", data)

## Process data from 3 pulses

We now want to repeat the reduction, but using more than a single pulse to improve our statistics.

We begin by loading the run with 3 pulses.

In [ ]:
folder = "../3-mcstas/SANS_with_sample_3_pulse"
sample = utils.load_sans(folder)
sample

In [ ]:
sample.hist(tof=200, y=200).plot(norm="log", vmin=1.0e-2)

We can see that we have 3 distinct pulses (3 horizontal bands of events).

In order to combine the data from the 3 pulses (also known as 'pulse folding'),
we need to compute a neutron time-of-flight relative to the start of each pulses, instead of absolute time.

### Exercise 4: fold the pulses

You are provided with a function that will perform the folding correctly,
assuming you supply it the correct inputs.

In [ ]:
help(utils.fold_pulses)

Instead of reading the bin edges off the figure above,
you should try to compute your bin edges in a way that is based on some known parameters of your simulation (e.g. wavelength range) as well as the physical layout of the instrument.

**Hints:**

- You have 3 pulses, so you need to supply 4 time-of-flight edges
- Assume the first pulse has a zero time offset
- The frequency of the ESS source is 14 Hz
- To convert neutron wavelength to speed, you can use: $~~~~v = \displaystyle\frac{h}{m_{\mathrm{n}}\lambda} $

**Solution:**

In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


In [ ]:
# Insert your solution:


## Save results to disk

Once again, we need to save our results to disk:

In [ ]:
data = folded_normed.copy()
data.coords["Q"] = sc.midpoints(data.coords["Q"])

save_xye("sans_iofq_3pulses.dat", data)